In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
class Optimize:
    
    def __init__(self, f, a, b):
        self.f = f
        self.a = a
        self.b = b
    
    def f_d(self, x):
        h = 0.0000000001
        return (self.f(x + h) - self.f(x)) / h
    
    def f_dd(self, x):
        h = 0.00001
        return (self.f(x + 2 * h) - 2 * self.f(x + h) + self.f(x)) / (h ** 2)
    
    def fib(self, n):  # n-ое число фибоначчи
        f1, f2 = 0, 1
        if n == 0:
            return f1
        if n == 1:
            return f2
        for i in range(n - 1):
            f1, f2 = f2, f1 + f2
        return f2
    
    def gold_ratio_search(self, eps=0.01):
        f = self.f
        a, b = self.a, self.b
        T = (3 - 5 ** 0.5) / 2
        x1 = a + T * (b - a)
        x2 = a + b - x1
        f1, f2 = f(x1), f(x2)
        i = 0
        while b - a > eps:
            i += 1
            if f1 <= f2:
                b = x2
                x2 = x1
                x1 = a + b - x2
                f1, f2 = f(x1), f1
            else:
                a = x1
                x1 = x2
                x2 = a + b - x1
                f1, f2 = f2, f(x2)
        return 1 / 2 * (a + b), f(1 / 2 * (a + b)), i
    
    def dichotomy_search(self, eps=0.01):  # метод дихотомии
        delta = eps / 2
        a, b = self.a, self.b
        x1 = (a + b - delta) / 2
        x2 = (a + b + delta) / 2
        i = 0
        while abs(a - b) > eps:
            if self.f(x1) < self.f(x2):
                b = x2
            else:
                a = x1
            x1 = (a + b - delta) / 2
            x2 = (a + b + delta) / 2
            i += 1
        if b < a:
            a, b = b, a
        return (b - a) / 2, self.f((b - a) / 2), i
    
    def fib_search(self, eps=0.01):  # метод фибоначчи
        a, b = self.a, self.b
        n = 0
        while (b - a) / eps > self.fib(n):
            n += 1
        x1 = a + self.fib(n - 2) / self.fib(n) * (b - a)
        x2 = a + self.fib(n - 1) / self.fib(n) * (b - a)
        k = 1
        i = 0
        while k < n - 2:
            if self.f(x1) > self.f(x2):
                a = x1
                x1 = x2
                x2 = a + self.f(n - k - 1) / self.f(n - k) * (b - a)
            else:
                b = x2
                x2 = x1
                x1 = a + self.f(n - k - 2) / self.f(n - k) * (b - a)
            k += 1
            i += 1
        x2 = x1 + eps
        if self.f(x1) < self.f(x2):
            b = x2
        else:
            a = x1
        return (b - a) / 2, self.f((b - a) / 2), i
            
        
    def cutting(self, eps=0.01):  # метод секущих
        f_d, f = self.f_d, self.f
        a, b = self.a, self.b
        i = 0
        if f_d(a) < 0 and f_d(b) > 0:
            z = b - f_d(b) * (b - a) / (f_d(b) - f_d(a))
            while abs(f_d(z)) > eps:
                if f_d(z) > 0:
                    a = z
                else:
                    b = z
                z = b - f_d(b) * (b - a) / (f_d(b) - f_d(a))
                i += 1
            return z, f(z), i
        return 0, f(0), i

    def tangent(self, eps=0.01):  # метод ньютона
        f_dd, f_d, f = self.f_dd, self.f_d, self.f
        a, b = self.a, self.b
        x = (a + b) / 2
        i = 0
        while abs(f_d(x)) > eps:
            x -= f_d(x) / f_dd(x)
            i += 1
        return x, f(x), i
    
    def get_all_methods(self, epses=[0.1, 0.001, 0.00001]):
        methods = [self.gold_ratio_search, self.dichotomy_search, self.fib_search, self.cutting, self.tangent]
        str_methods = ['gold ratio', 'dichotomy', 'fibonacci', 'cutting', 'newton'] 
        return np.array([[method(eps=eps)[2] for method in methods] for eps in epses]).T, str_methods

In [5]:
def f(x):
    return x ** 4

In [6]:
a, b = -3, 3
epsilons = [0.1, 0.001, 0.00001]
minimize = Optimize(f, a, b)
res, strs = minimize.get_all_methods(epses=epsilons)
df = pd.DataFrame(res, columns=[f"eps={eps}" for eps in epsilons])
df.index = strs
df

,eps=0.1,eps=0.001,eps=1e-05
gold ratio,9,19,28
dichotomy,7,14,21
fibonacci,8,17,27
cutting,0,0,0
newton,0,0,0
